In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

// TODO : Find out how to restart kernel

In [5]:
%%plantuml 

@startuml
Alice -> Bob: Authentication Request
Bob --> Alice: Authentication Response
@enduml 

UsageError: Cell magic `%%plantuml` not found.


In [44]:
# Delegation pattern example
class Microwave:
    def __init__(self):
        pass

    def heat_up_food(self):
        print("Food is being microwaved")

    def boil_water(self):
        print("Water is being boiled")

    def ruin_cake(self):
        print("Cake is being ruined... ah come on...why even lol")

class Dishwasher:
    def __init__(self):
        pass

    def wash_dishes(self):
        print("Dishwasher starting")

    def leave_spots(self):
        print("Dishes have spots, have fun being embarassed in front of guests")

    def wash_clothes(self):
        print("Washing dish shaped clothes... wtf")

class Kitchen ():
    def __init__(self):
        self.microwave = Microwave()
        self.dishwasher = Dishwasher()

        def get_custom_methods(obj):
            return [f for f in dir(obj) if not f.startswith('_') and callable(getattr(obj, f))]

        # how to unit test not callable?
        self.microwave_methods = get_custom_methods(self.microwave)
        # checking for callable overhead
        self.dishwasher_methods = get_custom_methods(self.dishwasher)

    def __getattr__(self, func):
        # what is the diff between *args and **kwargs?
        def method(*args):
            if func in self.microwave_methods:
                return getattr(self.microwave, func)(*args)
            if func in self.dishwasher_methods:
                return getattr(self.dishwasher, func)(*args)
            else:
                raise AttributeError(f"Method {func} not found")
        return method


In [45]:
kitchen = Kitchen()
kitchen.wash_dishes()
kitchen.boil_water()
kitchen.ruin_cake()
kitchen.leave_spots()
kitchen.wash_clothes()

Dishwasher starting
Water is being boiled
Cake is being ruined... ah come on...why even lol
Dishes have spots, have fun being embarassed in front of guests
Washing dish shaped clothes... wtf


In [1]:
# Decorator pattern examples
from datetime import datetime

class Employee():
    def __init__(self, name, salary):
        self.name = name
        self.salary = salary

class SalaryYTDReport:
    def __init__(self, employees):
        self.employees = employees
        self.data = []

    def prep_data(self):
        self.data = []
        for employee in self.employees:
            emp_sal_dict = {'Employee': employee.name, 'YTD_salary':self.calculate_YTD_salary(employee.salary)}
            self.data.append(emp_sal_dict)

    def calculate_YTD_salary(self, salary):
        current_month = datetime.today().month
        ytd_salary = (salary/12) * current_month
        ytd_salary = round(ytd_salary, 2)
        return ytd_salary

In [12]:
class HTMLReportDecorator:
    def __init__(self, report):
        self.report = report
        self.html_report = []
        self.report_methods = [f for f in dir(SalaryYTDReport) if not f.startswith('_') and callable(getattr(SalaryYTDReport, f))]
        self.report_attributes = [a for a in report.__dict__.keys()]

    def __getattr__(self, func):
        if func in self.report_methods:
            def method(*args):
                # for functions, return AND call the function
                return getattr(self.report, func)(*args)
            return method
        elif func in self.report_attributes:
            # for attributes, just return the attribute
            return getattr(self.report, func)
        else:
            raise AttributeError

    def report_data(self):
        self.html_report = []
        self.prep_data()
        for row in self.data:  # type: ignore
            name = f"<b>{row['Employee']}</b>"
            ytd = f"<i>{row['YTD_salary']}</i>"
            html_row = f"{name}: {ytd}<br />"
            self.html_report.append(html_row)

In [ ]:
empl1 = Employee('Mike', -47)
empl2 = Employee('Ike', 41)
empl3 = Employee('Scott', 38)
empl4 = Employee('Cathy', 44)

report = SalaryYTDReport([empl1, empl2, empl3, empl4])
report = HTMLReportDecorator(report)

for employee in report.employees:  # type: ignore
    print(employee.name)

report.report_data()
report.html_report


In [15]:
import time # use sleep() to report logging time

class PerformanceLogReportDecorator:
    def __init__(self, report):
        self.report = report
        self.report_methods = [f for f in dir(SalaryYTDReport) if not f.startswith('_') and callable(getattr(SalaryYTDReport, f))]
        self.report_attributes = [a for a in report.__dict__.keys()]

    def __getattr__(self, func):
        if func in self.report_methods:
            def method(*args):
                return self.log(func, *args)
            return method
        elif func in self.report_attributes:
            return getattr(self.report, func)
        else:
            raise AttributeError


    def log(self, func, *args):
        start = datetime.now()
        getattr(self.report, func)(*args)
        time.sleep(1) # putting this here to show logging is actually happening
        end = datetime.now()
        microseconds = (end - start).microseconds
        print(f"{func} took {microseconds} miroseconds to run")


In [16]:
empl1 = Employee('Mike', -47)
empl2 = Employee('Ike', 41)
empl3 = Employee('Scott', 38)
empl4 = Employee('Cathy', 44)

report = SalaryYTDReport([empl1, empl2, empl3, empl4])
report = PerformanceLogReportDecorator(report)
# in realworld, instead of printing to console; this would be written to a file or database for future analysis
report.prep_data()

prep_data took 5180 miroseconds to run


In [17]:
class Animal:
    def __init__(self, name, num_of_legs):
        self.name = name
        self.num_of_legs = num_of_legs
    
    def get_number_of_legs(self):
        return self.num_of_legs

class Dog(Animal):
    def __init__(self, name, num_of_legs):
        super().__init__(name, num_of_legs)

    def bark(self):
        print("woof woof")

In [ ]:
class Decorator:
    def __init__self(self, model):
        self.model = model
        self.model_methods = [f for f in dir(type(self.model)) if not f.startswith('_') and callable(getattr(type(self.model), f))]
        self.model_attributes = [a for a in self.model.__dict__.keys()]

    def __getattr__(self, func):
        if func in self.model_methods:
            def method(*args):
                # what is the difference between methods and functions?
                # - methods are associated with the instance of the class they belong to
                # - functions are not associated with any object instance, they can just be called by name
                # if the instance method is a delegate function, return AND call that delegate function
                return getattr(self.model, func)(*args)
            return method
        elif func in self.model_attributes:
            # what is the difference between attributes and methods?
            # - a method is a function that is defined in the class
            # - an attribute is an instance variable defined in the class
            # TODO : hmm, I'm not sure I understand this part
            # Are methods considered attributes?
            # - Methods are attributes. Everything in Python is objects, really, with methods and functions 
            # and anything with a __call__() method being callable objects. They are all objects that respond
            # to the () call expression syntax. Attributes then, are objects found by attribute lookup on 
            # other objects
            # AH HA??? is access to the SELF as FIRST argument the difference between a method and an attribute?
            # - Any variable that is bound in a class is a class attribute
            # - Methods receive an instance of the class, conventionally called self , as the first argument
            return getattr(self.model, func)
        else:
            raise AttributeError

    

### -- [EXPLAIN LIKE I'M 11] --


#### Explain `conda` and `pip`
***

Conda and pip are two different package managers for installing and managing packages/modules in Python. They have some overlap in functionality, but there are some key differences:

Conda

Conda is a language-agnostic package manager. It can install and manage packages for any language, not just Python.
Conda creates and manages environments. Conda environments are isolated from each other and from the base system.
Conda installs packages binaries (pre-compiled).
Conda was created specifically for Python programs, but it can package and distribute software for any language.
Conda as a package manager helps find and install packages.
Conda can install both conda packages and pip packages.
Conda is included in Anaconda distribution and Miniconda.

Pip

Pip is a package manager for Python packages, or modules if you like.
Pip does not create or manage environments. It uses the environments provided by conda or venv.
Pip installs Python packages from the Python Package Index (PyPI).
Pip cannot install conda packages, only Python packages.
Pip comes bundled with all versions of Python >= 2.7.9 and >= 3.4.

Some key points:

Conda is a language-agnostic package manager and environment manager. 
Pip is a package manager for Python packages. 
Conda can install both conda packages and pip packages. 
Pip can only install pip packages.

In practice:

Use conda to install any package or create/manage environments. 
Use pip to install Python packages within a conda environment. 
Never use pip to install packages into the base conda environment.

A visual summary:

Conda: Package Manager + Environment Manager
   |- Conda packages (language-agnostic)
   |- Pip packages (Python) 
Pip: Package Manager (Python)

So pip is suitable for Python projects, while conda is suitable for projects that use multiple languages. Conda is the preferred tool for installing and managing packages for Python when a package requires a specific Python version since it excels at creating and managing environments.
I hope this helps clarify the difference between these two important tools! Let me know if you have any other questions.

#### Explain the difference between `%pip install Bokeh` and `!pip install Bokeh` in Jupyter code cell
***

In a Jupyter Notebook or JupyterLab code cell, the `%` and `!` symbols are used to run special commands known as "magic commands" and shell commands, respectively.

1. `%pip install Bokeh`: This is a magic command that starts with a single `%` sign, which is used for line magics. Line magics are single-line commands that are interpreted by the IPython kernel, not the Python interpreter. In this case, `%pip` is a built-in magic command that runs the `pip` package manager within the context of the current Jupyter Notebook kernel. This command installs the Bokeh package in the same Python environment used by the current kernel.

2. `!pip install Bokeh`: This command starts with an exclamation mark `!`, which is used to run shell commands directly from the Jupyter Notebook cell. When you use an exclamation mark, the command is executed in a new shell process outside of the IPython kernel. In this case, the `!pip install Bokeh` command runs the `pip` package manager to install the Bokeh package. Depending on your system configuration and paths, this command may install the package in a different Python environment from the one used by your Jupyter Notebook kernel.

In most cases, both commands will install the Bokeh package as expected, and you can use it in your Jupyter Notebook. However, it's generally recommended to use the `%pip` magic command (`%pip install Bokeh`) to ensure that the package is installed in the same environment as the Jupyter Notebook kernel. This way, you can avoid potential issues related to multiple Python environments or configurations.